In [2]:
import pandas as pd
import json

In [3]:
dev_db = r'Dataset\Procesado\dev_db.parquet'

In [108]:
df = pd.read_parquet(dev_db)
df.sample(3)

,item_id,malo,neutral,positivo,developer,release_date,price
18014,240,0,0,1,Valve,2004-11-01,19.99
6218,333980,0,0,1,ACQUIRE Corp.,2015-05-26,29.99
23968,200210,0,0,1,Wild Shadow Studios,2012-02-20,0.00


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49057 entries, 0 to 49056
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   item_id       49057 non-null  int32         
 1   malo          49057 non-null  int64         
 2   neutral       49057 non-null  int64         
 3   positivo      49057 non-null  int64         
 4   developer     49057 non-null  object        
 5   release_date  49057 non-null  datetime64[ns]
 6   price         49057 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(3), object(1)
memory usage: 2.4+ MB


## <div style="color: lightgreen; text-align: center;"> .  def developer  . </div>

In [25]:
df1 = df.copy()

In [26]:
df1.sample(5)

,item_id,malo,neutral,positivo,developer,release_date,price
21034,394360,0,0,1,Paradox Development Studio,2016-06-06,39.99
45654,322170,0,1,0,RobTop Games,2014-12-22,3.99
37084,224260,1,0,0,No More Room in Hell Team,2011-10-31,0.00
28115,440,0,0,1,Valve,2007-10-10,0.00
14995,369400,0,0,1,Eforb,2015-08-27,0.00


In [28]:
df2 = df1[df1['developer'] == 'Valve']

In [41]:
list_x = sorted(df2['release_date'].dt.year.unique(), reverse=True)

In [42]:
print(list_x)

[2012, 2011, 2010, 2009, 2008, 2007, 2006, 2004, 2003, 2001, 2000, 1999, 1998]


In [51]:
lista_final =[]

In [52]:
for year in list_x:
    
    df_year = df2[df2['release_date'].dt.year == year]
    c_items = df_year['item_id'].value_counts().sum()
    free = df_year[df_year['price'] == 0].value_counts().sum()
    
    m = {year:[{'Cantidad de items':c_items},{'Contenido Free': free }]}
    lista_final.append(m)
    

In [53]:
print(lista_final)

[{2012: [{'Cantidad de items': 3759}, {'Contenido Free': 0}]}, {2011: [{'Cantidad de items': 374}, {'Contenido Free': 0}]}, {2010: [{'Cantidad de items': 126}, {'Contenido Free': 93}]}, {2009: [{'Cantidad de items': 752}, {'Contenido Free': 0}]}, {2008: [{'Cantidad de items': 33}, {'Contenido Free': 0}]}, {2007: [{'Cantidad de items': 3858}, {'Contenido Free': 3736}]}, {2006: [{'Cantidad de items': 18}, {'Contenido Free': 0}]}, {2004: [{'Cantidad de items': 486}, {'Contenido Free': 0}]}, {2003: [{'Cantidad de items': 4}, {'Contenido Free': 0}]}, {2001: [{'Cantidad de items': 1}, {'Contenido Free': 0}]}, {2000: [{'Cantidad de items': 67}, {'Contenido Free': 0}]}, {1999: [{'Cantidad de items': 17}, {'Contenido Free': 0}]}, {1998: [{'Cantidad de items': 62}, {'Contenido Free': 0}]}]


## <div style="color: lightgreen; text-align: center;"> .  def best_developer_year  . </div>


In [6]:
df.sample(5)

,item_id,malo,neutral,positivo,developer,release_date,price
21306,377160,0,0,1,Bethesda Game Studios,2015-11-09,29.99
15848,440,0,0,1,Valve,2007-10-10,0.00
35650,49520,0,1,0,"Gearbox Software,Aspyr (Mac &amp; Linux)",2012-09-17,19.99
2993,205100,1,0,0,Arkane Studios,2012-10-08,9.99
20527,4000,0,0,1,Facepunch Studios,2006-11-29,9.99


In [7]:
b_dev = df[df['release_date'].dt.year == 2014]

In [8]:
dev_reviews = b_dev.groupby('developer')['positivo'].sum()

In [14]:
dev_reviews_s = dev_reviews.sort_values(ascending=False)

In [44]:
pp = dev_reviews_s.to_dict()
ppp = list(pp.keys())

In [45]:
ppp[1]

'Coffee Stain Studios'

In [31]:
dev_x = list(dev_reviews_s.index)

In [32]:
print(type(dev_x))

<class 'list'>


In [24]:
dev_top3 = list(dev_reviews_s.head(3).index)

In [25]:
print(type(dev_top3))

<class 'list'>


In [22]:
dev_top3[0]

'Edge of Reality'

In [12]:
top_devs = list(dev_reviews.nlargest(3).keys())


In [13]:
print(top_devs[1])

Coffee Stain Studios


In [10]:
print(top_devs)

['Edge of Reality', 'Coffee Stain Studios', 'New World Interactive']


In [ ]:
top_dev_json = top_devs.to_json()

In [84]:
print(top_dev_json)

{"Smartly Dressed Games":632,"Freejam":217,"Studio Wildcard,Instinct Games,Efecto Studios,Virtual Basement LLC":188}


## <div style="color: lightgreen; text-align: center;"> .  def developer_reviews_analysis  . </div>

In [90]:
dev_d = df[df['developer'] == 'Valve']

In [91]:
dev_r = dev_d.groupby('developer')[['positivo', 'malo']].sum()

In [92]:
dev_j = dev_r.to_json()

In [ ]:
result = 

In [93]:
print(dev_j)

{"positivo":{"Valve":6056},"malo":{"Valve":1098}}


In [95]:
dev = 'Valve' # Variable event de lambda
d_data = df[df['developer'] == dev]

neg_r = d_data['malo'].sum()
pos_r = d_data['positivo'].sum()

result = {dev:{'Negativo': neg_r, 'Positivo': pos_r}}


result_df = pd.DataFrame(result).T  # Convertir el diccionario en un DataFrame transpuesto
result_json = result_df.to_json()

print(result)

{'Valve': {'Negativo': 1098, 'Positivo': 6056}}


## <div style="color: lightgreen; text-align: center;"> .  def user_data  . </div>

In [147]:
user_db = r'Dataset\Procesado\user_db.parquet'

In [148]:
df = pd.read_parquet(user_db)
df.head(5)

,price,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,...,Sports,Strategy,Utilities,Video Production,Web Publishing,item_id,user_id,review_sum,item_name,playtime_forever
0,9.99,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10,-GM-Dragon,2.0,Counter-Strike,14.0
1,9.99,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10,-Mad-,2.0,Counter-Strike,8.0
2,9.99,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10,-SEVEN-,9.0,Counter-Strike,617.0
3,9.99,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10,-SatansLittleHelper-,1.0,Counter-Strike,511.0
4,9.99,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10,-Ultrix,1.0,Counter-Strike,5.0


In [ ]:
# Optimizar  El dataframe solo para usuarios con un conteo mayor a 200

In [167]:
df3 = df[['user_id', 'item_id', 'price']]

In [179]:
df5 = df3.head(100000)

In [180]:
df5.to_parquet(r'Dataset\Procesado\user_def_2.parquet', index=False)

In [149]:
user = '-Mad-'

In [150]:
dfm = df[df['user_id'] == user]

In [151]:
money = dfm.groupby('user_id')['price'].sum()
items = dfm.groupby('user_id')['item_id'].value_counts().sum()

In [159]:
result = {"Usuario X": user, "Dinero gastado": float(money.values[0]), "Cantidad de items": int(items)}

In [160]:
print(result)

{'Usuario X': '-Mad-', 'Dinero gastado': 3638.23, 'Cantidad de items': 296}


In [ ]:
#Lambda Function

# user = '-Mad-'

# dfm = df[df['user_id'] == user]

# money = dfm.groupby('user_id')['price'].sum()
# items = dfm.groupby('user_id')['item_id'].value_counts().sum()

# result = {"Usuario X": user, "Dinero gastado": float(money.values), "Cantidad de items": int(items)}

# print(result)

## <div style="color: lightgreen; text-align: center;"> .  def UserForGenre  . </div>

In [130]:
df.sample(5)

,price,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,...,Sports,Strategy,Utilities,Video Production,Web Publishing,item_id,user_id,review_sum,item_name,playtime_forever
455071,24.99,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,219640,plumbers,4.0,Chivalry: Medieval Warfare,56.0
144442,14.99,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,16450,Lad123,2.0,F.E.A.R. 2: Project Origin,0.0
1876309,19.99,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,620,76561198055819674,5.0,Portal 2,184.0
1819482,19.99,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,50620,Thisbananaistastyfam,6.0,Darksiders,0.0
482013,9.99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,220780,76561198079765149,1.0,Thomas Was Alone,177.0


In [191]:
df_u = df.drop(columns=['price', 'review_sum','item_name']).copy()

In [198]:
df_u.head(3)

,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,Free to Play,...,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,item_id,user_id,playtime_forever
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,-GM-Dragon,14.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,-Mad-,8.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,-SEVEN-,617.0


In [196]:
df_u = df_u.head(50000)


In [197]:
df_u.to_parquet(r'Dataset\Procesado\user_def_1.parquet')

In [128]:
genre_x = "Action"

In [129]:
dfg = df[genre_x]

In [131]:
horas_j = dfm.groupby('user_id')['playtime_forever'].sum()

In [133]:
user_horas_j = horas_j.idxmax()

In [165]:
resp = {f"Usuario con más horas jugadas para {genre_x}": user_horas_j, "Total horas jugadas": float(horas_j.values[0])}

In [166]:
print(resp)


{'Usuario con más horas jugadas para Action': '-Mad-', 'Total horas jugadas': 175458.0}


In [ ]:
genre_x = "Action"

dfg = df[genre_x]

horas_j = dfm.groupby('user_id')['playtime_forever'].sum()
user_horas_j = horas_j.idxmax()

resp = {f"Usuario con más horas jugadas para {genre_x}": user_horas_j, "Total horas jugadas": float(horas_j.values[0])}

## <div style="color: lightgreen; text-align: center;"> .  def recomendacion_juego  . </div>

In [47]:
#importamos la biblioteca
from sklearn.metrics.pairwise import cosine_similarity

In [85]:
import numpy as np

In [48]:
# Eliminamos la columnas con las que no vamos a trabajar del df
df_processed = df.drop(columns=['developer', 'release_date', 'price']) 

In [49]:
df_processed.set_index('item_id', inplace=True)

In [87]:
df_processed.shape

(49057, 3)

In [50]:
# Cálculo de similitud
similarity_matrix = cosine_similarity(df_processed)

In [ ]:
if not producto.empty:
    product_index = producto.index[0]
    product_similarities = similarity_matrix[product_index]
    most_similar_products_indices = np.argsort(-product_similarities)
    most_similar_products = data.loc[most_similar_products_indices, 'nombre']
    print("Los productos más similares al producto", nombre_del_producto, "son:")
    print(most_similar_products)
else:
    print("Producto no encontrado")

In [88]:
item_id = 10

In [86]:
if not item_id.empty:
    
    product_similarities = similarity_matrix[df[df['item_id'] == item_id]]
    most_similar_products_indices = np.argsort(-product_similarities)
    most_similar_products = data.loc[most_similar_products_indices, 'nombre']
    print("Los productos más similares al producto", nombre_del_producto, "son:")
    print(most_similar_products)
else:
    print("Producto no encontrado")

MemoryError: Unable to allocate 17.9 GiB for an array with shape (1, 49057, 49057) and data type float64

In [99]:
def get_top_similar_items(item_id, n=5):
    item_index = df_processed.index.get_loc(item_id)
    similar_items = similarity_matrix[item_index]
    # Obtener los índices de los elementos más similares en orden descendente
    similar_items_indices = similar_items.argsort()[::-1][:n]
    # Convertir a una lista plana
    similar_items_indices = similar_items_indices.flatten()
    # Excluir el propio elemento y devolver los elementos similares del DataFrame
    return df_processed.iloc[similar_items_indices]

In [106]:
#Hago una prueba
recommended_items = get_top_similar_items(338000)

In [107]:
print(recommended_items.head(5))

         malo  neutral  positivo
item_id                         
200210      0        1         0
440         1        0         0
338000      1        0         0
274190      1        0         0
369580      0        1         0
